In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.053650617599487305
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 11.323316269823568
200000 17.965900605933832
300000 20.061487425770405
400000 26.408924571509175
500000 24.475433155277372
600000 28.296872664429365
700000 29.973826765800663
800000 32.2568968116709
900000 36.10417291432856
1000000 41.34461966239179
1100000 47.189536036804796
1200000 43.83988446700679
1300000 32.21749660310404
1400000 56.24457527845556
1500000 58.80686616581875
1600000 58.23305078475292
1700000 59.60524438607984
1800000 60.769703701616834
1900000 59.45746530418778
2000000 53.44356650746273
2100000 58.9148645843824
2200000 60.08246346138673
2300000 55.095709889998595
2400000 44.605322642502195
2500000 24.98156655265446
2600000 56.5913845208221
2700000 57.773996266495836
2800000 46.148151170630335
2900000 49.01955617677644
3000000 46.186405

29700000 51.952471092251436
29800000 48.226654946310205
29900000 49.676837525467974
30000000 45.037417126457875
30100000 25.83856979855925
30200000 36.421308911793886
30300000 44.60953926192288
30400000 48.960753359352424
30500000 46.623051447848425
30600000 50.2664724204696
30700000 46.851012967686195
30800000 51.45243832139387
30900000 48.7446865584929
31000000 51.65816092036562
31100000 23.076680013972044
31200000 47.326094875855325
31300000 50.46546518726088
31400000 55.069141664897884
31500000 44.31900535044686
31600000 26.211240449836996
31700000 53.95721397010845
31800000 51.76700267534315
31900000 54.12225339234875
32000000 50.57369887051179
32100000 54.22659550152759
32200000 49.23990678207015
32300000 50.73418112853934
32400000 46.66839332745557
32500000 29.518924933674498
32600000 34.10575517508296
32700000 48.37163531387147
32800000 49.51382208305141
32900000 51.1819512101967
33000000 50.41157891394264
33100000 50.417443183326476
33200000 51.80606694503596
33300000 54.13925

59800000 22.834199675137302
59900000 50.01441182620355
60000000 45.31637176766658
60100000 47.80901359319519
60200000 47.35659294398278
60300000 52.61046385790366
60400000 48.735474826097864
60500000 52.199392269277546
60600000 36.68958969092324
60700000 31.36967096407339
60800000 47.1819315256771
60900000 48.906975075188505
61000000 46.30410628414687
61100000 49.1383070018508
61200000 44.37159649780208
61300000 45.144845707772234
61400000 13.751896721169592
61500000 41.81259622404252
61600000 46.1629742684586
61700000 44.63910241469194
61800000 46.098949173767174
61900000 45.78350461659531
62000000 49.06865466194819
62100000 38.448045284153636
62200000 29.767176458624792
62300000 53.306244489657225
62400000 49.26761453107762
62500000 51.685518882637474
62600000 49.43626721813915
62700000 48.804617550642526
62800000 47.43960362780391
62900000 48.864498200468915
63000000 45.649871655470896
63100000 21.237280659243645
63200000 43.830254981424034
63300000 46.45302630947726
63400000 42.042

89900000 39.4770811747224
90000000 44.681497646993996
90100000 39.21221253359491
90200000 41.49244441725213
90300000 12.628113724517515
90400000 40.737391152014176
90500000 43.37025177148127
90600000 39.1575480885219
90700000 42.4536122918676
90800000 20.638569963066026
90900000 36.59353807146443
91000000 45.60740719493018
91100000 41.9580165982651
91200000 47.90322506501026
91300000 43.64483097813525
91400000 45.79174400139053
91500000 44.5127138699416
91600000 47.3185024662023
91700000 43.17435537966618
91800000 45.77050282178138
91900000 41.377228049049194
92000000 44.45491470130195
92100000 34.26897195315725
92200000 19.482522371456643
92300000 41.613879615476435
92400000 37.22184245740832
92500000 42.454289529487646
92600000 39.462648959917594
92700000 9.266127365822879
92800000 43.77367073627348
92900000 38.25814833949165
93000000 40.62146323276844
93100000 38.26266996802267
93200000 18.1561374028006
93300000 41.16569253947166
93400000 40.690751874388155
93500000 42.5987162456732

119300000 39.43220857070629
119400000 35.278612152505204
119500000 19.544116769099357
119600000 39.361789599229375
119700000 37.88179350469594
119800000 39.196102153660924
119900000 39.09879042618556
120000000 42.949340399791204
120100000 41.07236027546029
120200000 44.9818725097054
120300000 41.571768415668856
120400000 42.73283021183821
120500000 40.45305434803694
120600000 42.65631707426683
120700000 38.6632961932136
120800000 40.163632670521835
120900000 38.38715752902583
121000000 39.441894597064696
121100000 36.951920265499304
121200000 27.798703783288296
121300000 25.00336111685936
121400000 34.766416607028155
121500000 27.7191889994341
121600000 22.079963348712564
121700000 34.86442745179424
121800000 31.46851930212805
121900000 23.83911554655894
122000000 36.85942023470184
122100000 39.55167366275967
122200000 35.96202859205315
122300000 37.29512290225321
122400000 40.224914728279614
122500000 40.501005812762926
122600000 42.95963952204643
122700000 39.79589282848458
122800000

...

494200000 7.258469323000424
494300000 7.60970443678729
494400000 6.862073641605502
494500000 5.135024862862052
494600000 6.174673065875673
494700000 7.9117427959567985
494800000 7.707512390429777
494900000 7.879507274990936
495000000 8.012761888814953
495100000 3.5280750581520044
495200000 8.196598166851615
495300000 7.859253107530085
495400000 7.55294403616813
495500000 7.510312822132946
495600000 6.461242331664726
495700000 6.48195431249099
495800000 5.723769673877623
495900000 7.859286903459222
496000000 7.4987550288932034
496100000 7.8971426096488235
496200000 7.606490762439979
496300000 3.7439402984844015
496400000 7.644499534214867
496500000 7.9132538413762425
496600000 7.338349680450912
496700000 7.380839701374825
496800000 5.553579345430352
496900000 6.839169140304399
497000000 5.78909834342358
497100000 8.03230280366854
497200000 7.3633006251844
497300000 7.702157559896935
497400000 7.459205683594826
497500000 3.8983567807045336
497600000 7.142113027305488
497700000 7.90234969

523500000 5.91307451789805
523600000 5.808777308656997
523700000 5.764109292668183
523800000 5.997869584179299
523900000 4.361741677061038
524000000 2.820762288573281
524100000 3.603286736815538
524200000 5.450447763070978
524300000 5.785472181074285
524400000 5.607783247566535
524500000 5.89106889705869
524600000 5.637484823449565
524700000 5.590242871868782
524800000 5.648792534039172
524900000 5.248228220860382
525000000 5.737988875055726
525100000 4.315008791965133
525200000 2.89396899772203
525300000 3.5614852052744634
525400000 5.182460201309646
525500000 5.217943201901937
525600000 5.602210232020628
525700000 5.4435713968575055
525800000 5.767523546768714
525900000 5.932510373396049
526000000 5.748157038050434
526100000 5.751089221350837
526200000 5.696691023051385
526300000 4.440031571144897
526400000 1.9937935299244622
526500000 5.292707277155124
526600000 6.979452929373429
526700000 5.388908956310439
526800000 5.860441214052513
526900000 5.759032385908096
527000000 6.62963528

552500000 3.2839602322787442
552600000 3.769995620380181
552700000 4.196440992320431
552800000 3.7304956387125414
552900000 2.7454796772547043
553000000 4.156907156633618
553100000 4.374681970641583
553200000 3.1214274408215563
553300000 4.453104882211626
553400000 4.665196680777423
553500000 4.725477355013746
553600000 4.355160981449294
553700000 2.891217487091823
553800000 4.159505603916111
553900000 4.059543479657428
554000000 3.7720054331698956
554100000 2.6145721430591053
554200000 4.060917213480881
554300000 4.156459388023859
554400000 3.1969769689922507
554500000 4.455613901434611
554600000 4.551006676543486
554700000 4.605823918254563
554800000 4.547667570580984
554900000 2.5456185727600693
555000000 4.411128672416213
555100000 4.143578916971833
555200000 3.6951459919148406
555300000 2.442006835787791
555400000 4.03039458408938
555500000 4.113348430884171
555600000 2.978550162607727
555700000 4.623989398001288
555800000 4.526997153184079
555900000 4.4268062171798475
556000000 4

580100000 2.491238084998315
580200000 2.0657651642301063
580300000 2.003121271880914
580400000 2.085781078356904
580500000 2.105559087799214
580600000 2.6682192176419655
580700000 2.814484930693223
580800000 1.78743415961133
580900000 1.9785984717394802
581000000 1.9794918464675846
581100000 1.7647915644786893
581200000 2.5904103796294757
581300000 2.732391970763997
581400000 1.7021065223906306
581500000 1.8842866939015712
581600000 1.973374535997492
581700000 1.9040008774331907
581800000 2.716548496155411
581900000 2.740940034063685
582000000 1.7156991854934711
582100000 2.0400711945807894
582200000 1.9528217654212097
582300000 1.7348925237118682
582400000 2.6724177825792688
582500000 2.3119133326820815
582600000 1.5317597723354195
582700000 1.853741755027258
582800000 1.910539567086959
582900000 1.7300472187031963
583000000 2.654738184001678
583100000 2.5336491067628106
583200000 1.7474695588079487
583300000 2.1097542215782092
583400000 1.5203022014538488
583500000 1.8195512700022036

608500000 1.065652996053889
608600000 0.7308857386447947
608700000 0.7023981008962287
608800000 0.625090146237223
608900000 0.5841852971091093
609000000 0.5717622881553842
609100000 0.7744660229165415
609200000 0.419675317232418
609300000 0.9216267567384614
609400000 1.085246307667858
609500000 0.7438474529547076
609600000 0.5166730276045243
609700000 0.8139254694167211
609800000 0.43980974386621713
609900000 0.9204002136230249
610000000 1.1427967781674895
610100000 0.5650225700039446
610200000 0.509099342136893
610300000 0.7404776320360568
610400000 0.5466738087862552
610500000 0.8836498332592487
610600000 0.9669489452886046
610700000 0.5082037351034701
610800000 0.5917269803931275
610900000 0.7992298596014765
611000000 0.5449995551154495
611100000 0.6548545164072971
611200000 0.863243269229016
611300000 0.3532662441393872
611400000 0.5591041342207279
611500000 0.8154686349257462
611600000 0.5702797707469046
611700000 0.7009530749522064
611800000 0.6022034828056362
611900000 0.5962804